In [ ]:
include("init.jl")
using TensorOperations
using TensorDecompositions
using Statistics
using LinearAlgebra
import Plots
import StatsPlots
import LightGraphs

# Tensor decompositions sort of explained for Colin

This little notebook is for me to play with the tensor packages in Julia; to try to remember some stuff from my degree; and to help Colin understand what the point of it all is.

First things first - there is a bug in Julia where `zeroes` is misspelt.

In [ ]:
zeroes = zeros

## Generating our tensor

The idea behind a tensor decomposition or factorisation is that there's a lot of degeneracy around in the world these days.

Say, for example, you generate a tensor from three vectors. Then, by definition, that tensor can be described by three vectors rather than having to describe the whole tensor.

The point of this is that the three vectors are more interesting than the tensor - they will describe the patterns that have led to that tensor being created.

Mathematically:

$$ a_{ijk} = \Sigma_{r=1}^R u_{ir} v_{jr} w_{kr} $$

where $R$ is the number of factors asked for in the algorithm.

In this notebook, we'll generate a tensor $a$ from $u,v,w$, and use a tensor decomposition to attempt to retrieve those vectors.

In [ ]:
DIMS = [1000,200,30,3]
u,v,w,x = [rand(dim) for dim in DIMS]
z = rand(DIMS...)
a = zeroes(DIMS...)
@show a |> size

# This is using Einstein's summation convention - the whole tensor a is generated by iterating through all possible pairs 
@tensor a[i,j,k,l] = u[i] * v[j] * w[k] * x[l]# + 0.001*z[i,j,k]
@show mean(a)

In [ ]:
#F = candecomp(a, 1, (randn(10,1),randn(20,1),randn(30,1));
#    # NB: need to have 2D arrays for the guess; 1D isn't good enough
#    compute_error=true,
#    method=:ALS,
#    maxiter=100_000,
#);
F = nncp(a, 1;
    compute_error=true,
    maxiter=10,
);
F.factors

In [ ]:
# Conversely, tensor operations wants 1D arrays
U,V,W,X = [f |> Iterators.flatten |> collect for f in F.factors]

In [ ]:
@tensor A[i,j,k,l] := U[i] * V[j] * W[k] * X[l];

In [ ]:
@show mean(abs.(A-a))
@show mean(a)
@show mean(A)
inds = rand(1:minimum(DIMS))
@show A[inds...]
@show a[inds...]
@show maximum(abs.(A-a))

# Nice. Easy-mode.
# NB: u,v,w aren't the same as U,V,W; but a and A are very similar

In [ ]:
StatsPlots.plot([StatsPlots.boxplot(vec,legend=:none) for vec in (u,U,v,V,w,W)]..., layout=(1,6))

# This is quite interesting: it gets the shapes of the distributions right, but the normalisation wrong.

## So what does this mean for graphs?

First, let's make an adjacency matrix that varies with time.

We make an undirected graph with no self-loops as our base.

In [ ]:
N = 50
TIMESTEPS = 10
DENSITY = 0.05 # unused atm
LINK_HIATUS_RATE = 0.1
#base_adj = Int.(rand(N,N) .< DENSITY) |> Symmetric
#base_adj = base_adj .& mod.(one(base_adj).+1,2) # get rid of self-loops;

# Let's make an actually interesting graph
#g = LightGraphs.barabasi_albert(N,1)
#g = LightGraphs.erdos_renyi(N,DENSITY)
g = LightGraphs.watts_strogatz(N,4,0.01)
base_adj = g |> LightGraphs.adjacency_matrix;

At each timestep, we temporarily turn off LINK_HIATUS_RATE links, and make a nice tensor.

In [ ]:
import ProgressMeter

In [ ]:
perms = Int.(rand(N,N,TIMESTEPS) .> LINK_HIATUS_RATE)
for t in 1:TIMESTEPS
    perms[:,:,t] = perms[:,:,t]  |> Symmetric
end

# Want to make a more recognisable temporal pattern
# Mask a block off and turn it on and off at various times
 perms = zeroes(Int,N,N,TIMESTEPS)
SPECIAL_NODES = N ÷ 10
SPECIAL_TIME = 3
for i in 1:N, j in 1:N, t in 1:TIMESTEPS
    perms[i,j,t] = i <= SPECIAL_NODES && j <= SPECIAL_NODES && t <= SPECIAL_TIME ? 0 : 1
end
# should probably find a nicer way of doing this...

ADJ = zeroes(N,N,TIMESTEPS) # making this takes _AGES_
for i in 1:N, j in 1:N, t in 1:TIMESTEPS
    ADJ[i,j,t] = base_adj[i,j] & perms[i,j,t]
end

#ProgressMeter.@showprogress for t in 1:TIMESTEPS
#    ADJ[:,:,t] = base_adj
#end

# I expected
# @tensor ADJ[i,j,t] = base_adj[i,j] & perms[i,j,t]
# to work, but it doesn't.

ADJ |> size

In [ ]:
#methods(nncp)

In [ ]:
R = 10
@time F = nncp(ADJ, R; # R is the number of components ≈ number of communities
    # I think Ciro spoke about optimising this number somewhere
    # papers refer to it - core consistency
    compute_error=true,
    maxiter=1000,
);
# (each u*v*w[:,r] is a component; each of u,v,w is a factor)
# n = 1000 -> 2 seconds
# n = 10,000 -> bloody ages. severely limited by i/o - using 25% CPU only. has been like, 5 minutes. Am bored.

In [ ]:
F.factors[3] |> size

In [ ]:
F.factors[1] # The largest numbers in each nodes correspond to the nodes most active in that community
(u,v,w) = [f for f in F.factors]

In [ ]:
Plots.plot(F.factors[3]) # You can see that some communities are inactive for the first few timesteps, as we'd expect

In [ ]:
# Find the 5 bestest nodes?
nbest = (n,v) ->  [findfirst(x->x==f,v) for f
        in sort(v,rev=true)[1:n]
]
@show nbest(5,u[:,1])
@show nbest(5,v[:,1])

# Nodes that have high scores in the first vector of the first factor
# Tend to link to nodes with high scores in the second vector of the
# first factor
# and tend to link to things at high-scoring times in the third vector

# Which sounds weirdly bi-partite?

# So, a "community" is this:
# Nodes which link to similar nodes
# Nodes which have most of their links turned on at high-scoring times

In [ ]:
Plots.heatmap(u)

In [ ]:
import GraphPlot
import Colors

In [ ]:
community_size = N ÷ R
colours = Colors.distinguishable_colors(R,Colors.colorant"blue")
nodefillarr = []
for n in 1:N
    ind = findmax(u[n,:])[2]
    push!(nodefillarr,colours[ind])
end
GraphPlot.gplot(g;nodefillc=nodefillarr) # Sure, these kind of look like communities.
# Eugh, if you put too many nodes in, GraphPlot gives up trying to colour them
# Would be nice to colour nodes by strength of their association too.

In [ ]:
colours # Key - 1:R+1 from left to right (last is uncategorised)

In [ ]:
nodefillarr |> unique

## Network interpretation

Say you have a temporal adjacency matrix which you wish to decompose into $R$ communities:

$$ a_{ijt} = \Sigma_{r=1}^R u_{ir} v_{jr} w_{tr} $$

NB: Each of $u,v,w$ is roughly (self) column-wise orthogonal - i.e, if an entry is large in one column of, e.g, u, then it will be smaller in another column of u.

Therefore, for any edge $a_{ijt}$ to appear from node $j$ to $i$ at time $t$, it must be true that, for some $r$, $u_{ir} v_{jr} w_{tr} \approx 1$.

Ignoring the temporal factor for a moment, we can say that high-scoring nodes in $\boldsymbol{u}_{r}$ are probably linked to by nodes in $\boldsymbol{v}_{r}$ - all at once, i.e, each $r$ fuzzily denotes a community of nodes who link to each other: so, $u_{ir}$ denotes the strength

$w_{tr}$ has a similar meaning: the highest scoring times for each $r$ means that the high-scoring nodes in $u$ and $v$ are "turned on" and have more outgoing or incoming links at those times.

In an undirected graph, we expect $u$ and $v$ to look similar.

## Fiddling

You don't need to pay attention to stuff below this line.

In [ ]:
b = u * v'
@tensor a[i,j,k] = b[i,j] * w[k] + 0.001*z[i,j,k]

fixme = ones(1:TIMESTEPS)
@tensor testADJ[i,j,t] := base_adj[i,j] * fixme[t] + perms[i,j,t]

# I really don't understand the difference between these expressions
# Or why fixme is needed

In [ ]:
@show findfirst(x->false,[1])

# Sparse playground

In [ ]:
using SparseArrays

DIMS = [10,20,30]
u,v,w = [round.(rand(dim)) for dim in DIMS] .|> hcat .|> SparseMatrixCSC # rounding should actually be done later
z = rand(DIMS...)
a = zeroes(DIMS...) # |> sparse # Error - sparse tensors aren't in Julia. Consider adopting https://github.com/JuliaTensors/SparseTensors.jl ?
# Or we could write a wrapper for taco - https://github.com/tensor-compiler/taco
@show a |> size

# This is using Einstein's summation convention - the whole tensor a is generated by iterating through all possible pairs 
#@tensor a[i,j,k] = u[i] * v[j] * w[k]# + 0.001*z[i,j,k]
for i in 1:DIMS[1], j in 1:DIMS[2], k in 1:DIMS[3]
    a[i,j,k] = u[i] * v[j] * w[k]
end
@show mean(a)

In [ ]:
base_adj |> Array

In [ ]:
times = []
Ns = 10 .^(1:0.1:4) .|> round .|> Int
R = 10
for n in Ns
    ADJ = zeroes(n,n,1) # making this takes _AGES_
    g = LightGraphs.watts_strogatz(n,4,0.01)
    base_adj = g |> LightGraphs.adjacency_matrix;
    ADJ[:,:,1] = base_adj

    t = @elapsed F = nncp(ADJ, 1; # R is the number of components ≈ number of communities
        # I think Ciro spoke about optimising this number somewhere
        # papers refer to it - core consistency
        compute_error=true,
        maxiter=100,
    );
    push!(times,t)
end

In [ ]:
Plots.scatter(Ns,times;
    xlabel = "N",
    ylabel = "Time taken / seconds",
    legend = :none,
    yscale = :log10,
    xscale = :log10
)

# tl;dr: power-law, N^k, where k is quite big

## Choosing R

https://github.com/alessandrobessi/corcondia/blob/master/coreconsistency.py <- CONCORDIA

In [ ]:
import BenchmarkTools
delta = δ(args...) = reduce(==,args) |> Int

In [ ]:
BenchmarkTools.@benchmark delta(2,2)

In [ ]:
# CONCORDIA
c = 0
for i in 1:N, j in 1:N, t in 1:TIMESTEPS
    temp = sum([u[i,r] + v[j,r] + w[t,r] for r in 1:R])
    c += (temp - δ(i,j,t))^2
end
c

In [ ]:
U = []
V = []
Σ = []
for m in [u,v,w]
    svds = svd(m)
    push!(U,svds.U)
    push!(V,svds.V)
    push!(Σ,svds.S |> Diagonal)
end
rADJ = zeroes(N,N,TIMESTEPS)
for i in 1:N, j in 1:N, t in 1:TIMESTEPS
    rADJ[i,j,t] = u[i] * v[j] * w[t]
end
rADJ |> size

In [ ]:
kron(transpose.(U)..., rADJ[:])